In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [12]:
## Load the imdb dataset word index

word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [13]:
# Load the pre-trained model with ReLU activation

model = load_model('Simple_rnn_imdb.h5')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.get_weights()

[array([[ 2.7567619e-01,  3.7834376e-01,  8.0537826e-01, ...,
         -2.0311539e-01, -2.5332221e-01, -2.7349275e-01],
        [ 3.4734942e-02,  1.1802245e-01,  1.7347965e-02, ...,
         -3.0077633e-04, -3.1999063e-02, -1.2028045e-01],
        [-9.6163817e-02, -5.2623883e-02,  1.2087180e-02, ...,
          1.0999306e-02,  5.9396159e-02, -4.4121325e-02],
        ...,
        [-4.5557927e-02, -8.7039024e-02,  2.7351573e-02, ...,
         -8.9571342e-02, -5.2307785e-02,  8.2658023e-02],
        [-9.9165581e-02,  8.0704600e-02, -6.5805599e-02, ...,
          3.5300322e-02, -4.0482592e-02,  4.6602771e-02],
        [ 5.2672992e-03, -5.7422981e-02, -1.8152785e-01, ...,
         -7.8445487e-02, -8.4794365e-02,  7.1334071e-02]], dtype=float32),
 array([[-0.01320327, -0.0425518 ,  0.04002448, ..., -0.03057784,
          0.05380711,  0.013091  ],
        [-0.01809673,  0.08801669,  0.05396733, ...,  0.02347291,
         -0.17219453,  0.00466892],
        [ 0.05757026, -0.06966183,  0.05437609

In [15]:
# Step 2: Helper Functions
# Function to decode reviews

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to process user input

def preprocess_text(text, maxlen=500, vocab_size=10000):
    words = text.lower().split()
    # Map words -> index (use 2 if not in vocab)
    encoded_review = [word_index.get(w, 2) for w in words if word_index.get(w, 2) < vocab_size]
    # Shift indices by +3 only ONCE (IMDB convention)
    encoded_review = [i + 3 for i in encoded_review]
    # Add "start" token = 1
    encoded_review = [1] + encoded_review
    # Pad/truncate
    padded_review = sequence.pad_sequences([encoded_review], maxlen=maxlen)
    return padded_review

In [16]:
### Prediction function

def predict_sentiment(review):
    preprocessed_input = preprocess_text(review)
    prediction = model.predict(preprocessed_input)
    sentiment = 'Positive' if prediction[0][0]> 0.5 else 'Negative'
    return sentiment, prediction[0][0]

In [34]:
# Step 4: User Input and prediction

# Example reveiw for prediction
example_review = "This movie was great and The acting was great and the plot was thrilling"

sentiment, score = predict_sentiment(example_review)

print(f'Review:{example_review}')
print(f'Sentiment:{sentiment}')
print(f'Prediction score:{score}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Review:This movie was great and The acting was great and the plot was thrilling
Sentiment:Positive
Prediction score:0.7891339659690857


In [30]:
for w in ["good", "fantastic", "nice"]:
    print(w, word_index.get(w))


good 49
fantastic 774
nice 324


In [31]:
tests = [
    "this movie was fantastic",
    "this movie was nice",
    "this movie was good",
    "this movie was terrible",
    "this was the worst movie"
]

for t in tests:
    s, p = predict_sentiment(t)
    print(f"{t} -> {s} ({p:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
this movie was fantastic -> Negative (0.4780)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
this movie was nice -> Positive (0.5373)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
this movie was good -> Positive (0.6990)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
this movie was terrible -> Negative (0.0364)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
this was the worst movie -> Negative (0.0077)


In [20]:
for layer in model.layers:
    print(layer.name, layer.get_weights()[0].shape if layer.get_weights() else "no weights")


embedding (10000, 128)
simple_rnn (128, 128)
dense (128, 1)
